# 0.0 Preparation

In [7]:
pip install pandas --upgrade

  Using cached pandas-2.2.3-cp312-cp312-macosx_10_9_x86_64.whl.metadata (89 kB)
Using cached pandas-2.2.3-cp312-cp312-macosx_10_9_x86_64.whl (12.5 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [60]:
pip install graphviz


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install pymc

  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.4 MB/s eta 0:00:00
Using cached cachetools-5.5.2-py3-none-any.whl (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 3.9 MB/s eta 0:00:00a 0:00:01m
  Created wheel for cons: filename=cons-0.4.6-py3-none-any.whl size=9094 sha256=edef41dc91e6be1ba5bbd07ad377ec629b7fd565a78a28b14ad4027be173264b
  Stored in directory: /Users/xanvelaa/Library/Caches/pip/wheels/91/0c/f7/7ba4ffbf0db6bf272bee59789131f6ec2d26966307a3054267
  Created wheel for logical-unification: filename=logical_unification-0.4.6-py3-none-any.whl size=13912 sha25

In [9]:
pip install openpyxl


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [52]:
# Import Pandas 
import pandas as pd

## Set view to two decimal points only 
pd.set_option('float_format', '{:2f}' .format) 

# Import NumPy
import numpy as np

# Import from statsmodel
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA

# Import pyMC
import pymc as pm


# 1.0 Modeling Tourist Data

In [14]:
df = pd.read_excel("/Users/xanvelaa/Downloads/Boracay_Weather_Tourist.xlsx", sheet_name = "tourist")
df.dtypes

date             datetime64[ns]
tourist_count             int64
dtype: object

In [18]:
final_df = (
    df
    .assign(
        month = lambda x: x["date"].dt.month,
        year = lambda x: x["date"].dt.year
    )
    [["month", "year", "tourist_count"]]
)

final_df.head()

,month,year,tourist_count
0,1,2022,35799
1,2,2022,80882
2,3,2022,150597
3,4,2022,186751
4,5,2022,201368


In [20]:
# Goal: predict values from 2025

model_df = (
    final_df
    .pivot_table(
        values='tourist_count',  # Pivot to get total sales as values
        index='month',        # Set 'week' as rows
        columns='year',      # Set 'year' as columns
    )
    .reset_index()
)

model_df

year,month,2022,2023,2024
0,1,35799.000000,177860.000000,172913.000000
1,2,80882.000000,160772.000000,174364.000000
2,3,150597.000000,184970.000000,191326.000000
3,4,186751.000000,213736.000000,182647.000000
4,5,201368.000000,207512.000000,205246.000000
5,6,193650.000000,196960.000000,183755.000000
6,7,183096.000000,207696.000000,159826.000000
7,8,157338.000000,178589.000000,121975.000000
8,9,122373.000000,124491.000000,145021.000000
9,10,135252.000000,139008.000000,162451.000000


In [142]:
# Initialize dictionary to store forecasts
forecasts = {}

# Loop through each month and fit an ARIMA model
for i, month in enumerate(model_df["month"]):
    # Get the time series for the current month (2022, 2023, 2024 values)
    ts = model_df.iloc[i, 1:].values  

    # Fit ARIMA model (p, d, q can be tuned)
    model = ARIMA(ts, order=(1, 1, 1))  
    model_fit = model.fit()
    
    # Forecast 1 step ahead (2025 value)
    forecast = model_fit.forecast(steps=1)[0]
    
    # Store the forecasted value
    forecasts[month] = forecast

# Convert forecasts into a DataFrame
forecast_df = (
    pd.DataFrame(

        list(forecasts.items()), 
        columns=["month", "Forecast_2025"]
        )
        .assign(Forecast_2025 = lambda x: x["Forecast_2025"].astype(int))
        )

forecast_df

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likel

,month,Forecast_2025
0,1,146323
1,2,161727
2,3,192503
3,4,193003
4,5,206013
5,6,190545
6,7,167744
7,8,124315
8,9,156059
9,10,175847


In [179]:
tourist_forecast = (
    pd.merge(
        model_df,
        forecast_df,
        on = "month", 
        how = "left"

    )
    .rename(columns = {'Forecast_2025':2025})
)

tourist_forecast

,month,2022,2023,2024,2025
0,1,35799.000000,177860.000000,172913.000000,146323
1,2,80882.000000,160772.000000,174364.000000,161727
2,3,150597.000000,184970.000000,191326.000000,192503
3,4,186751.000000,213736.000000,182647.000000,193003
4,5,201368.000000,207512.000000,205246.000000,206013
5,6,193650.000000,196960.000000,183755.000000,190545
6,7,183096.000000,207696.000000,159826.000000,167744
7,8,157338.000000,178589.000000,121975.000000,124315
8,9,122373.000000,124491.000000,145021.000000,156059
9,10,135252.000000,139008.000000,162451.000000,175847


# 2.0 Predicting Weather Data

In [107]:
weather_df = pd.read_excel("/Users/xanvelaa/Downloads/Boracay_Weather_Tourist.xlsx", sheet_name = "weather")
weather_df = weather_df[["date", "days_rained", "days_cloudy", "days_sunny"]]
weather_df.head()

,date,days_rained,days_cloudy,days_sunny
0,2021-01-01,25,4,2
1,2021-02-01,16,6,6
2,2021-03-01,13,11,7
3,2021-04-01,9,9,12
4,2021-05-01,7,14,10


In [108]:
weather_df_transformed = (
    weather_df
    .assign(
        month = lambda x: x["date"].dt.month,
        year = lambda x: x["date"].dt.year
    )
    [["month", "year", "days_rained", "days_cloudy", "days_sunny"]]
    .sort_values(by=["month", "year"], ignore_index=True)

)

In [ ]:
# Store predictions
predictions = []

# Get unique months in the dataset
months = weather_df_transformed["month"].unique()

# Bayesian model loop for each month
for month in months:
    # Filter data for the current month
    test = weather_df_transformed.query(f"month == {month}")
    days_in_this_month = test.iloc[0][["days_rained", "days_cloudy", "days_sunny"]].sum()


    # Get observed counts and years
    observed_counts = test[["days_rained", "days_cloudy", "days_sunny"]].values
    years = test["year"].values
    k = observed_counts.shape[1]  # Number of weather categories

    # Define coordinates for PyMC
    coords = {"year": years, "weather": ["rainy", "cloudy", "sunny"]}

    with pm.Model(coords=coords) as model_dm:
        # Dirichlet prior for weather proportions
        frac = pm.Dirichlet("frac", a=np.ones(k), dims="weather")

        # Lognormal prior for concentration parameter
        conc = pm.Lognormal("conc", mu=1, sigma=1)
        
        # Dirichlet-Multinomial probabilities per year
        p = pm.Dirichlet("p", a=frac * conc, dims=("year", "weather"))
        
        # Multinomial likelihood
        counts = pm.Multinomial(
            "counts", n=days_in_this_month, p=p, observed=observed_counts, dims=("year", "weather")
        )

        # Sample from posterior
        trace = pm.sample(1000, tune=500, chains=2, return_inferencedata=True, cores=1)

    # Predict for 2025
    with model_dm:
        future_posterior = pm.sample_posterior_predictive(trace, var_names=["p"])

    # Extract predicted probabilities for 2025
    pred_p = future_posterior.posterior_predictive["p"].mean(dim=["chain", "draw"]).sel(year=2024)

    # Convert probabilities to predicted counts for a 31-day month
    pred_counts = (pred_p * days_in_this_month).round().astype(int)

    # Store results
    predictions.append({
        "month": month,
        "year": 2025,
        "days_rained": pred_counts.sel(weather="rainy").values.item(),
        "days_cloudy": pred_counts.sel(weather="cloudy").values.item(),
        "days_sunny": pred_counts.sel(weather="sunny").values.item()
    })

Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [frac, conc, p]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 2 chains for 500 tune and 1_000 draw iterations (1_000 + 2_000 draws total) took 8 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics
Sampling: [p]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [frac, conc, p]


...

Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [frac, conc, p]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 2 chains for 500 tune and 1_000 draw iterations (1_000 + 2_000 draws total) took 8 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics
Sampling: [p]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

    month  year  days_rained  days_cloudy  days_sunny
0       1  2025           17            8           6
1       2  2025           13            9           6
2       3  2025            7           14          10
3       4  2025            9           11          10
4       5  2025           11           11           9
5       6  2025           18            8           4
6       7  2025           20            8           3
7       8  2025           18            9           4
8       9  2025           19            8           3
9      10  2025           18           10           3
10     11  2025           14           11           5
11     12  2025           15           11           5


In [170]:
# Convert predictions to DataFrame
weather_predictions = pd.DataFrame(predictions)

weather_forecast = (
    weather_df_transformed
    .merge(
        weather_predictions,
        on = ["month", "year", "days_rained", "days_cloudy", "days_sunny"],
        how = "outer",
    )
    .sort_values(by=["year", "month"], ignore_index=True)
    .query("year != 2021")
)

# 3.0 Finalize Training Data for LLM Fine Tuning

In [181]:
# Convert wide format to long format
tourist_predictions = (
    tourist_forecast
    .melt(id_vars=["month"], 
          var_name="year", 
          value_name="tourists")

    .assign(tourists = lambda x: x['tourists'].astype(int))
)

tourist_predictions.head()

,month,year,tourists
0,1,2022,35799
1,2,2022,80882
2,3,2022,150597
3,4,2022,186751
4,5,2022,201368


In [185]:
train_data = (
    pd
    .merge(
        weather_forecast,
        tourist_predictions,
        on = ["year", "month"],
        how = "outer"
    )
    .query("year == 2025")
    .fillna(0)
    .reset_index(drop=True)
)

train_data

/var/folders/9b/wdj2nc2n1xs_4gffmhjwvngw0000gn/T/ipykernel_13306/4063396499.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(0)


,month,year,days_rained,days_cloudy,days_sunny,tourists
0,1,2025,17,8,6,146323
1,2,2025,13,9,6,161727
2,3,2025,7,14,10,192503
3,4,2025,9,11,10,193003
4,5,2025,11,11,9,206013
5,6,2025,18,8,4,190545
6,7,2025,20,8,3,167744
7,8,2025,18,9,4,124315
8,9,2025,19,8,3,156059
9,10,2025,18,10,3,175847


In [ ]:
train_data.to_csv("train_data.csv")